## Einfacher Service Mesh inkl. Verschlüsselung
Dieses Notebook enthält die notwendigen Befehle zur Bereitstellung eines einfachen Service Mesh mit Istio.


### Namespace erstellen und Istio aktivieren

In [ ]:
%%bash
kubectl create namespace ms-rest
kubectl label namespace ms-rest istio-injection=enabled


Verschlüsselung des Datenverkehrs erzwingen

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: security.istio.io/v1beta1
kind: PeerAuthentication
metadata:
  name: default
  namespace: ms-rest
spec:
  mtls:
    mode: STRICT
EOF

### Deployment der Microservices

In [ ]:
%%bash
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/2.1.0-pod/catalog-pod.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/2.1.0-pod/customer-pod.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/2.1.0-pod/order-pod.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/2.1.0-pod/webshop-pod.yaml


### Bereitstellung der Services

In [ ]:
%%bash
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/catalog-service.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/customer-service.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/order-service.yaml
kubectl apply --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/webshop-service.yaml


### Status der Deployments und Services überprüfen

In [ ]:
%%bash
kubectl get pod,services --namespace ms-rest

Prüfen ob die Verschlüsselung aktiviert wurde

In [ ]:
%%bash
kubectl get peerauthentication -n ms-rest

### Webshop-URL ermitteln

In [ ]:
%%bash
echo "https://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ms-rest webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop


### Lasttest durchführen

In [ ]:
%%bash
# LastTest mit 404
URL="https://localhost":$(kubectl get service --namespace ms-rest webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop
hey -z 10s -c 50  ${URL}/order/order

Der Traffic sollte in Kiali, Jaeger oder Zipkin sichtbar sein.

### Aufräumen

In [ ]:
%%bash
kubectl delete namespace ms-rest

### Quellen
- [Sourcecode](https://gitlab.com/ch-mc-b/autoshop-ms/app/shop/-/tree/v2.1.0?ref_type=heads)
- [Kubernetes Deklarationen](https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates)
- [Container Registry](https://gitlab.com/ch-mc-b/autoshop-ms/app/shop/container_registry)